Import lib

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px

from sklearn.linear_model import LogisticRegression


ModuleNotFoundError: No module named 'plotly'

Import phishing_site_urls dataset

In [17]:
urls_df = pd.read_csv(r'phishing_site_urls.csv')

urls_df.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [18]:
urls_df.tail()

,URL,Label
549341,23.227.196.215/,bad
549342,apple-checker.org/,bad
549343,apple-iclods.org/,bad
549344,apple-uptoday.org/,bad
549345,apple-search.info,bad


In [19]:
urls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549346 entries, 0 to 549345
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549346 non-null  object
 1   Label   549346 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB


In [20]:
urls_df.isnull().sum()

URL      0
Label    0
dtype: int64

In [22]:
label_counter = pd.DataFrame(urls_df.Label.value_counts())

In [26]:
#fig = px.bar(label_counter, x=label_counter.index, y=label_counter.Label)